<a href="https://colab.research.google.com/github/oilportrait/test_colab/blob/main/solutionHW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/yeonmin/solution의 솔루션을 공부하기 위해서 주석을 달았습니다.

# 대회 데이터가 일부 변경되었습니다.
# 아래 경로의 Competition Site에 오시면 비슷한 데이터로 실습하실 수 있습니다.<br>
https://www.kaggle.com/competitions/original-instant-gratification

In [ ]:

import warnings #파이썬에서 경고메세지를 컨트로 하는 모듈
warnings.filterwarnings('ignore') # 발생하는 모든 경고 메세지를 무시

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook # 주피터 노트북에서 진행줄을 보여주는데 사용

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import StratifiedKFold

from sklearn.decomposition import KernelPCA
from sklearn.mixture import GaussianMixture as GMM
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import lightgbm as lgbm

In [ ]:
#데이터의 히스토그램을 표현하기 위한 클래스
class hist_model(object):

    # 클래스의 생성자 정의, 객체는 bins라는 매개변수를 가짐
    def __init__(self, bins=50):

        # bins: 히스토그램을 만들떄 사용할 구간
        self.bins = bins

    # fit은 주어진  데이터셋 X에 대해서 히스토그램을 계산
    def fit(self, X):

        # hight: 히스토그램의 높이 edge:이스토그램 구간 경계
        bin_hight, bin_edge = [], []

        # X.T로 데이터셋을 trnaspose 시킨다. 그렇게 해서 기존 X의 열을 기준으로 데이터를 var에게 전달한다.
        for var in X.T:

            # var에 들어간 데이터를 self.bins에 등록된 숫자를 기준으로 히스토그램을 계산한다.
            # bh: 각 구간에 속하는 데이터 포인트 수(빈도)를 나타낸다
            # bedge: 구간의 경계를 나타내는 리스트를 표현 ex) [0, 1, 2, 3] -> [0,1), [1,2), [2,3)
            bh, bedge = np.histogram(var, bins=self.bins)

            # bh,bedge를 기존에 정의한 배열에 저장한다.
            # bin_edge는 2차원 배열로 각 행은 X의 열 속성이며 각 행에 들어간 1차원 배열은 전체구간을 나누는 경계
            # bin_hight는 2차원 배열로 각 행은 X의 열 속성이며 각 행에 들어간 1차원 배열은 각 구간의 데이터 분포
            bin_hight.append(bh)
            bin_edge.append(bedge)

        # 객체의 속성을 데이터의 모든 데이터수와 구간을 numpy 배열로 변환하여 지정한다.
        self.bin_hight = np.array(bin_hight)
        self.bin_edge = np.array(bin_edge)

    # X에 대해서 점수를 예측하는 메소드 정의
    def predict(self, X):

        scores = []
        # X의 행을 기준으로 obs에 전달
        for obs in X:
            obs_score = []

            # i는 X의 열 속성이며 var은 그 속성의 데이터 값이다.
            for i, var in enumerate(obs):

                # (var > self.bin_edge[i]): 데이터 값이 구간의 경계들에 비교해서 큰지 알아보고 불리안 값을 가진 배열을 내보낸다.
                #.argmin(): 불리안 배열에서 처음으로 False가 나타난 위치의 인덱스를 반환
                # bin_num: self.bin_edge[i]에서 구간에 시작점의 배열 인덱스를 저장한다.
                bin_num = (var > self.bin_edge[i]).argmin()-1

                # 해당되는 데이터가 있는 구간의 높이를 배열에 append한다.
                obs_score.append(self.bin_hight[i, bin_num])

            # 결국 scores 행렬은 X의 행에서 각 데이터들이 자신이 속산 열의 속성에서 위치한 곳의 높이를 더하고 평균한 것들의 배열이다.
            # scores[0]은 X의 첫번쨰 행에 대한 score이다.
            scores.append(np.mean(obs_score))

        # score는 X 데이터의 행(관측치)에 대해서 전체 데이터셋과 분포적 차이가 큰지 작은지를 나타낸것이다.
        # 만약에 score가 크다면 관측치가 전체 데이터 분포에 가깝다는 의미
        # 이 점수로 이상치를 탐색하거나 데이터의 질을 알아볼 수 있을것으로 추측된다.
        return np.array(scores)

In [ ]:
#머신 모델을 실행하는 함수를 정의한다.
#clf_list: 분류기 리스트, train: 훈련 데이터셋, test: 테스트 데이터셋,
#random_state: 램덤성 정의, gmm_init_params: 가우시안 혼합모델 초기화 파마리터-> 초기 가우시안 분포 설정을 위한 내용
def run_model(clf_list, train, test, random_state, gmm_init_params='kmeans'):

    # 분류기 리스트의 길이를 계산해서 모델 개수를 설정
    MODEL_COUNT = len(clf_list)

    # len()으로 데이터셋의 길이로 행을 가지고 열은 MODEL_COUNT의 수로 초기화한다.
    # 배열의 모든 값은 0으로 초기화한다.
    oof_train = np.zeros((len(train), MODEL_COUNT))
    oof_test = np.zeros((len(test), MODEL_COUNT))

    # 훈련에 사용할 column을 결정
    #train의 열에서 'id', 'target', 'wheezy-copper-turtle-magic'을 제외한 열을 지정한 배열을 저장한다.
    train_columns = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]

    # tqdm_notebook은 반복문의 진행상황을 시각적으로 나타내며 for문은 저 함수를 제외한것과 의미가 같다.
    for magic in tqdm_notebook(range(512)):

        # wheezy-copper-turtle-magic열의 값과 magic이 일치하는 행을 선택
        x_train = train[train['wheezy-copper-turtle-magic'] == magic]
        x_test = test[test['wheezy-copper-turtle-magic'] == magic]
        print("Magic: ", magic, x_train.shape, x_test.shape)

        # 원본 데이터의 정보를 저장한다
        train_idx_origin = x_train.index
        test_idx_origin = x_test.index

        # train_columns에 해당하는 열들에 대한 표준편차를 계산한다.
        train_std = x_train[train_columns].std()

        # 표준편차가 2보다 큰 열들만을 선택한다.
        #train_std.index.values[]는 명시된 위치에 해당하는 열의 이름을 가져온다.
        cols = list(train_std.index.values[np.where(train_std >2)])

        # x_train 데이터 프레임의 인덱스를 리셋한다.
        # drop=True는 기존 인덱스를 열로 추가하지 않는다는 옵션
        x_train = x_train.reset_index(drop=True)

        # x_train의 target열을 분리해서 저장한다.
        y_train = x_train.target

        # x_train과 x_test 데이터셋을 cols에 해당하는 열들로 재구성한다.
        # .values는 선택된 열들로 구성된 데이터프레임을 넘파이 배열로 변환
        x_train = x_train[cols].values
        x_test = x_test[cols].values

        # 훈련 및 테스트 데이터를 수직으로 쌓아 하나의 데이터셋으로 만듬
        all_data = np.vstack([x_train, x_test])

        # Kernel PCA를 사용하여 데이터 차원을 축소
        # n_components: 유지할 주성분 수, kernel: 사용할 커널의 종류, random_state: 난수 생성 시드
        all_data = KernelPCA(n_components=len(cols), kernel='cosine', random_state=random_state).fit_transform(all_data)

        # 가우시안 혼합모델을 데이터에 적용한다.
        # n_components: 몇개의 가우시안 분포를 사용할지 정함, max_iter: 수렴하기 위한 최대 반복횟수, init_params: 가우시안 혼합모델 초기화 파마리터
        gmm = GMM(n_components=5, random_state=random_state, max_iter=1000, init_params=gmm_init_params).fit(all_data)

        # GMM을 이용해서 예측확률을 계산한다.
        # predict_proba: 데이터 포인트가 각 가우시안 구성요소에 속할 확률 계산
        gmm_pred = gmm.predict_proba(all_data)

        # GMM 점수를 계산한다. -> 모델이 얼마나 데이터를 설명하는지 알려주는 척도
        # score_samples: 데이터 포인터에 대한 log likelihood 계산
        gmm_score = gmm.score_samples(all_data).reshape(-1, 1)

        # GMM 레이블 예측
        # predict: 각 데이터가 가장 가능성이 높은 가우시안 구성요소에 속하는지를 결정 및 반환
        gmm_label = gmm.predict(all_data)

        # 히스토그램 기반특성을 데이터에 적용
        # hist_model 클래스의 인스턴스 생성
        hist = hist_model()

        # 데이터셋에 히스토그램 모델 학습
        hist.fit(all_data)

        # 히스토그램 모델을 이용해서 예측을 수행, 각 데이터가 각 속성의 히스토그램 구간에 얼마나 잘맞는지 계산(점수)
        # predict의 결과를 열 벡터로 재구성
        hist_pred = hist.predict(all_data).reshape(-1, 1)

        # all_data에 gmm_pred, hist_pred, gmm_score 배열의 여러 복사본을 추가한다.
        all_data = np.hstack([all_data, gmm_pred, gmm_pred, gmm_pred, gmm_pred, gmm_pred])
        all_data = np.hstack([all_data, hist_pred, gmm_score, gmm_score, gmm_score])

        # 데이터를 표준화한다.
        all_data = StandardScaler().fit_transform(all_data)

        # all_data를 훈련 부분과 테스트 부분으로 나눈다.
        x_train = all_data[:x_train.shape[0]]
        x_test = all_data[x_train.shape[0]:]

        # corss-validation을 위한 객체를 생성
        # n_splits: 데이터를 몇부분으로 나눌지 정의
        fold = StratifiedKFold(n_splits=5, random_state=random_state)

        # corss-validation 실행
        # gmm_label을 통해서 cross-validation의 타겟 레이블을 조각내서 반환시킨다.
        for trn_idx, val_idx in fold.split(x_train, gmm_label[:x_train.shape[0]]):

            # 분류기 목록을 처리하고 데이터에 적용한뒤 이후에 예측이 결합될 예정이다. 이는 스태킹의 기본 개념이다.
            # 분류기마다 for문을 만복한다.
            for model_index, clf in enumerate(clf_list):

                # 분류기를 데이터릐 훈련 인덱스로 지정한 부분으로 훈련시킨다.
                clf.fit(x_train[trn_idx], y_train[trn_idx])

                # validation에 대해서 예측 확률을 가져와서 저장한다.
                oof_train[train_idx_origin[val_idx], model_index] = clf.predict_proba(x_train[val_idx])[:,1]

                # 2023/03/02 데이터의 형식이 변경되어, x_test 예측 시 오류 발생하는 것 수정
                if x_test.shape[0] == 0:
                    continue

                #print(oof_test[test_idx_origin, model_index].shape)
                #print(x_test.shape)
                #print(clf.predict_proba(x_test)[:,1])
                # 테스트 데이터셋에 대한 예측 확률을 누적하고, 폴드 수로 나누어 평균을 계산한다.
                oof_test[test_idx_origin, model_index] += clf.predict_proba(x_test)[:,1] / fold.n_splits

    # 각 분류기의 성능을  roc, auc 점수로 평가
    for i, clf in enumerate(clf_list):
        print(clf)
        print(roc_auc_score(train['target'], oof_train[:, i]))
        print()

    # 예측 결과를 데이터 프레임 형태로 변환해서 저장 후 반환
    oof_train_df = pd.DataFrame(oof_train)
    oof_test_df = pd.DataFrame(oof_test)

    return oof_train_df, oof_test_df

In [ ]:
# ../input/instant-gratification/위치에서 모든 파일과 하위 디렉토리의 이름을 리스트로 저장
os.listdir('../input/instant-gratification/')

['sample_submission.csv', 'train.csv', 'test.csv']

In [ ]:
# 훈련,테스트 데이터셋을 지정한다.
train = pd.read_csv('../input/instant-gratification/train.csv')
test = pd.read_csv('../input/instant-gratification/test.csv')

In [ ]:
# 여러 머신모델의 파라미터 설정을 정의한다.
# SVM의 파라미터를 지정한다.
# probability: 확률 추정을 활성화할지 설정, kernel: 사용할 커널의 종류 지정, degree: 다항식 커널의 차수를 지정
# gamma: 커널 계수를 지정, nu: 오류의 상한과 서포트 벡트의 하한을 지정, coef0: 독립항의 계수 지정
svnu_params = {'probability':True, 'kernel':'poly','degree':4,'gamma':'auto','nu':0.4,'coef0':0.08, 'random_state':4}
svnu2_params = {'probability':True, 'kernel':'poly','degree':2,'gamma':'auto','nu':0.4,'coef0':0.08, 'random_state':4}

# Quadratic Discriminant Analysis의 파라미터를 지정한다.
# reg_param: 정규화 파라미터 지정
qda_params = {'reg_param':0.111}

# Support Vector Classifier의 파라미터를 지정한다.
svc_params = {'probability':True,'kernel':'poly','degree':4,'gamma':'auto', 'random_state':4}

# K-NN의 파라미터를 지정한다.
# n_neighbors: 이웃의 수 지정
neighbor_params = {'n_neighbors':16}

# Logistic Regression의 파라미터를 지정한다.
# solver: 최적화에 사용할 알고리즘 지정, penalty: 사용할 정규화 종류 지정, C: 정규화 강도 설정
lr_params = {'solver':'liblinear','penalty':'l1','C':0.05,'random_state':42}

In [ ]:
# 여러 머신러닝을 초기화 한다.
nusvc_model = svm.NuSVC(**svnu_params)
nusvc2_model = svm.NuSVC(**svnu2_params)
qda_model = QuadraticDiscriminantAnalysis(**qda_params)
svc_model = svm.SVC(**svc_params)
knn_model = neighbors.KNeighborsClassifier(**neighbor_params)
lr_model = linear_model.LogisticRegression(**lr_params)

# 초기화 한 모델을 리스트에 저장
model_list = [nusvc_model, nusvc2_model, qda_model, svc_model, knn_model, lr_model]

# 모델들의 여러 시나리오에 대한 예측결과를 저장한다.
oof_train_kmeans_seed1, oof_test_kmeans_seed1 = run_model(model_list, train, test, 1)
oof_train_kmeans_seed2, oof_test_kmeans_seed2 = run_model(model_list, train, test, 2)
oof_train_random_seed1, oof_test_random_seed1 = run_model(model_list, train, test, 1, 'random')
oof_train_random_seed2, oof_test_random_seed2 = run_model(model_list, train, test, 2, 'random')

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

Magic:  0 (534, 258) (0, 257)
Magic:  1 (510, 258) (0, 257)
Magic:  2 (495, 258) (0, 257)
Magic:  3 (538, 258) (0, 257)
Magic:  4 (516, 258) (0, 257)
Magic:  5 (521, 258) (0, 257)
Magic:  6 (518, 258) (0, 257)
Magic:  7 (506, 258) (0, 257)
Magic:  8 (522, 258) (0, 257)
Magic:  9 (528, 258) (0, 257)
Magic:  10 (498, 258) (0, 257)
Magic:  11 (482, 258) (0, 257)
Magic:  12 (496, 258) (0, 257)
Magic:  13 (485, 258) (0, 257)
Magic:  14 (486, 258) (0, 257)
Magic:  15 (516, 258) (0, 257)
Magic:  16 (517, 258) (0, 257)
Magic:  17 (511, 258) (0, 257)
Magic:  18 (518, 258) (0, 257)
Magic:  19 (526, 258) (0, 257)
Magic:  20 (512, 258) (0, 257)
Magic:  21 (532, 258) (0, 257)
Magic:  22 (537, 258) (0, 257)
Magic:  23 (522, 258) (0, 257)
Magic:  24 (495, 258) (0, 257)
Magic:  25 (491, 258) (0, 257)
Magic:  26 (489, 258) (0, 257)
Magic:  27 (504, 258) (0, 257)
Magic:  28 (537, 258) (0, 257)
Magic:  29 (498, 258) (0, 257)
Magic:  30 (515, 258) (0, 257)
Magic:  31 (509, 258) (0, 257)
Magic:  32 (483, 2

In [ ]:
# 각 시나리오들의 모든 예측 결과의 평균을 구한다.
# 여기서 서로 다른 모델들의 예측 결과를 평균화해서 결합했다.
# 이로서 개별모델의 강점을 포착하여 보다 강력한 전체예측을 도출할 수 있고 이는 스태킹 기법의 핵심 개념이고 목적이다.
train_second = (oof_train_kmeans_seed1 + oof_train_kmeans_seed2 + oof_train_random_seed1 + oof_train_random_seed2)/4
test_second = (oof_test_kmeans_seed1 + oof_test_kmeans_seed2 + oof_test_random_seed1 + oof_test_random_seed2)/4

# 앙상블 성능 평가
# roc_auc_score는 훈련 타겟 레이블과 예측 결과의 평균 사이의 ROC AUC 점수를 계산
print('Ensemble', roc_auc_score(train['target'], train_second.mean(1)))

Ensemble 0.9658510356167946


In [ ]:
# LightGBM의 파라미터를 지정
lgbm_meta_param = {
        #'bagging_freq': 5,
        #'bagging_fraction': 0.8,
        'min_child_weight':6.790, # 자식노드를 분할하기 위한 데이터의 최소 가중치 합
        "subsample_for_bin":50000, # 히스토그램 빈을 구축할때 사용한 샘플 수
        'bagging_seed': 0, # 배깅을 위한 난수 시드
        'boost_from_average':'true', # 평균값 부스팅 여부
        'boost': 'gbdt',  # 부스팅 타입 지정
        'feature_fraction': 0.450, # 각 반복에서 사용할 특성의 비율
        'bagging_fraction': 0.343, # 각 데이터에서 베깅을 위한 데이터 비율
        'learning_rate': 0.025, # 학습률 지정
        'max_depth': 10, # 트리의 최대 깊이 지정
        'metric':'auc', # 성능평가 메트릭 지정
        'min_data_in_leaf': 78, # 하나의 리프에 있어야할 최소 데이터수
        'min_sum_hessian_in_leaf': 8, # 리프에서 요구하는 헤시안의 최소합
        'num_leaves': 18, # 하나의 트리가 가질수 있는 최대 잎의 수
        'num_threads': 8, # 병렬 처리에 사용할 스레드 수
        'tree_learner': 'serial', # 트리 학습 방식
        'objective': 'binary', # 학습 목표
        'verbosity': 1, # 출력 메세지의 상세도 조정
        'lambda_l1': 7.961, # L1정규화 항
        'lambda_l2': 7.781 # L2정규화 항
        #'reg_lambda': 0.3,
    }

# MLP의 파라미터를 지정
# activation: 활성 함수, solver: 최적화를 사용할 알고리즘, tol: 최적화의 허용 오차
# hidden_layer_sizes: 은닉층 크기를 설정
mlp16_params = {'activation':'relu','solver':'lbfgs','tol':1e-06, 'hidden_layer_sizes':(16, ), 'random_state':42}

In [ ]:
# 반복할 랜덤 시드 수
SEED_NUMBER = 4

# fold의 수를 지정
NFOLD = 5

# 훈련 데이터의 타겟 변수 설정
y_train = train['target']

# 예측 결과를 저장할 배열을 초기화 시킨다.
oof_lgbm_meta_train = np.zeros((len(train), SEED_NUMBER))
oof_lgbm_meta_test = np.zeros((len(test), SEED_NUMBER))
oof_mlp_meta_train = np.zeros((len(train), SEED_NUMBER))
oof_mlp_meta_test = np.zeros((len(test), SEED_NUMBER))

# 랜덤 시드에 따른 반복 실행
for seed in range(SEED_NUMBER):
    print("SEED Ensemble:", seed)
    mlp16_params['random_state'] = seed
    lgbm_meta_param['seed'] = seed

    # 폴드를 적용하기 위해서 세팅
    folds = StratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=seed)

    # 각 폴드에 대한 아래의 과정 수행
    for fold_index, (trn_index, val_index) in enumerate(folds.split(train_second, y_train), 1):
        print(f"{fold_index} FOLD Start")

        # 훈련 데이터와 validation 데이터를 설정
        trn_x, trn_y = train_second.iloc[trn_index], y_train.iloc[trn_index]
        val_x, val_y = train_second.iloc[val_index], y_train.iloc[val_index]

        # MLP모델을 훈련
        mlp_meta_model = neural_network.MLPClassifier(**mlp16_params)
        mlp_meta_model.fit(trn_x, trn_y)

        # validation과 test 데이터에 대한 예측 데이터를 저장한다.
        oof_mlp_meta_train[val_index, seed] = mlp_meta_model.predict_proba(val_x)[:,1]
        oof_mlp_meta_test[:, seed] += mlp_meta_model.predict_proba(test_second)[:,1]/NFOLD
        print("MLP META SCORE: ", roc_auc_score(val_y, oof_mlp_meta_train[val_index, seed]))

        # lgbm에 대한 데이터셋을 준비한다.
        dtrain = lgbm.Dataset(trn_x, label=trn_y, silent=True)
        dcross = lgbm.Dataset(val_x, label=val_y, silent=True)

        # lgbm에 대한 훈련을 진행한다.
        lgbm_meta_model = lgbm.train(lgbm_meta_param, train_set=dtrain, valid_sets=[dtrain, dcross],
                                     verbose_eval=False, early_stopping_rounds=100)

        # 모델에 따른 예측 확률을 계산하고 저장한다.
        oof_lgbm_meta_train[val_index, seed] = lgbm_meta_model.predict(val_x)
        oof_lgbm_meta_test[:, seed] += lgbm_meta_model.predict(test_second)/NFOLD
        print("LGBM META SCORE: ", roc_auc_score(val_y, oof_lgbm_meta_train[val_index, seed]))

SEED Ensemble: 0
1 FOLD Start
MLP META SCORE:  0.9679149157241265
LGBM META SCORE:  0.967647530635912
2 FOLD Start
MLP META SCORE:  0.9680669908384355
LGBM META SCORE:  0.9674288751838224
3 FOLD Start
MLP META SCORE:  0.968603479583272
LGBM META SCORE:  0.9683328443489624
4 FOLD Start
MLP META SCORE:  0.9686502145269664
LGBM META SCORE:  0.9682168798889179
5 FOLD Start
MLP META SCORE:  0.9686257557500182
LGBM META SCORE:  0.968488858766663
SEED Ensemble: 1
1 FOLD Start
MLP META SCORE:  0.9694567291526788
LGBM META SCORE:  0.9689072412786429
2 FOLD Start
MLP META SCORE:  0.9684907321825804
LGBM META SCORE:  0.967816212999968
3 FOLD Start
MLP META SCORE:  0.9676346435556265
LGBM META SCORE:  0.9671898995884282
4 FOLD Start
MLP META SCORE:  0.9694087333144282
LGBM META SCORE:  0.9689640788253138
5 FOLD Start
MLP META SCORE:  0.9686281947259526
LGBM META SCORE:  0.9679262404257606
SEED Ensemble: 2
1 FOLD Start
MLP META SCORE:  0.967660503571859
LGBM META SCORE:  0.9673343320936969
2 FOLD S

In [ ]:
# 기존에 저장한 예측 결과를 데이터 프레임으로 변환 시키고 그 데이터 프레임에 평균을 계산해서 넣는다.
oof_lgbm_meta_train_df = pd.DataFrame(oof_lgbm_meta_train).mean(axis=1).to_frame().rename(columns={0:'lgbm'})
oof_lgbm_meta_test_df = pd.DataFrame(oof_lgbm_meta_test).mean(axis=1).to_frame().rename(columns={0:'lgbm'})
oof_mlp_meta_train_df = pd.DataFrame(oof_mlp_meta_train).mean(axis=1).to_frame().rename(columns={0:'mlp'})
oof_mlp_meta_test_df = pd.DataFrame(oof_mlp_meta_test).mean(axis=1).to_frame().rename(columns={0:'mlp'})

In [ ]:
# 기존에 많은 모델에 대해서 계산 한 예측 결과들을 합친다.
oof_train_third = pd.concat([train_second, oof_lgbm_meta_train_df, oof_mlp_meta_train_df], axis=1)
oof_test_third = pd.concat([test_second, oof_lgbm_meta_test_df, oof_mlp_meta_test_df], axis=1)

# oof_train_third.mean(1)는 데이터 프레임의 각 행에 대해 열의 평균을 구한다.
# 열의 평귱과 타겟 데이터를 비교해서 roc,auc 점수를 계산
print('Ensemble', roc_auc_score(train['target'], oof_train_third.mean(1)))

Ensemble 0.9671407107739252


In [ ]:
# 예전에 계산한 여러 평균값들을 합쳐서 데이터 셋을 형성
oof_train_third = pd.concat([train_second, oof_lgbm_meta_train_df, oof_mlp_meta_train_df], axis=1)
oof_test_third = pd.concat([test_second, oof_lgbm_meta_test_df, oof_mlp_meta_test_df], axis=1)

# roc,auc 점수를 계산
print('Ensemble', roc_auc_score(train['target'], oof_train_third.mean(1)))

Ensemble 0.9671407107739252


In [ ]:
submission = pd.read_csv('../input/instant-gratification/sample_submission.csv')

# 예측 결과를 추가시킨다.
submission["target"] = oof_test_third.mean(1)

# 제출 파일을 저장한다.
submission.to_csv("submission.csv", index=False)